# Anime Dataset 2023

Dataset Kaggle : https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset
- anime-dataset-2023.csv
- user-filtered.csv

The Goal of this project is to recommand anime for a specific user considering his anime list.

In [1]:
from cornac.models import ItemKNN, UserKNN
from cornac.data.dataset import Dataset

import scipy.sparse as sparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.neighbors import NearestNeighbors


/home/wyvern/.local/share/virtualenvs/PROJET-NeHBW21p/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IMPORT_PATH_BASE = "datasets/"

## Loading the datasets

In [3]:
# Read the anime dataset
anime_df = pd.read_csv(IMPORT_PATH_BASE + 'anime-dataset-2023.csv')

# Read the user dataset
user_df = pd.read_csv(IMPORT_PATH_BASE + 'user-filtered.csv')

In [4]:
anime_df

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1931/...


## Filtering the data

In [5]:
anime_df.drop(columns=['Other name', 'Name', 'Synopsis', 'Source', 'Premiered', 'Status', 'Producers', 'Licensors', 'Duration'], inplace=True)
anime_df.rename(columns={'English name': 'Name'}, inplace=True)
anime_df = anime_df.drop(anime_df[anime_df.eq('UNKNOWN').any(axis=1)].index)
anime_df = anime_df[anime_df['Type'].isin(['Movie', 'TV', 'TV Short'])]
user_df = user_df[user_df['anime_id'].isin(anime_df['anime_id'])]
anime_df = anime_df[anime_df['anime_id'].isin(user_df['anime_id'])]


In [6]:
anime_df

,anime_id,Name,Score,Genres,Type,Episodes,Aired,Studios,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Sunrise,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi",Movie,1.0,"Sep 1, 2001",Bones,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Madhouse,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,"Jul 3, 2002 to Dec 25, 2002",Sunrise,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,"Sep 30, 2004 to Sep 29, 2005",Toei Animation,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20915,48470,D_Cide Traumerei the Animation,5.81,"Action, Drama, Fantasy",TV,13.0,"Jul 10, 2021 to Oct 2, 2021",SANZIGEN,PG-13 - Teens 13 or older,9647.0,4243,102,6258.0,23808,https://cdn.myanimelist.net/images/anime/1635/...
20916,48471,Irina: The Vampire Cosmonaut,7.31,"Fantasy, Sci-Fi",TV,12.0,"Oct 4, 2021 to Dec 20, 2021",Arvo Animation,PG-13 - Teens 13 or older,2459.0,1499,689,49075.0,140342,https://cdn.myanimelist.net/images/anime/1393/...
20921,48488,Higurashi: When They Cry – SOTSU,6.73,"Avant Garde, Horror, Mystery, Supernatural, Su...",TV,15.0,"Jul 1, 2021 to Sep 30, 2021",Passione,R - 17+ (violence & profanity),5201.0,1882,722,41407.0,104539,https://cdn.myanimelist.net/images/anime/1083/...
20922,48491,Encouragement of Climb: Next Summit,7.63,"Adventure, Slice of Life",TV,12.0,"Oct 5, 2022 to Dec 21, 2022",8bit,PG-13 - Teens 13 or older,1307.0,4347,90,5556.0,22465,https://cdn.myanimelist.net/images/anime/1942/...


In [7]:
user_df

,user_id,anime_id,rating
0,0,67,9
1,0,6702,7
2,0,242,10
3,0,4898,0
5,0,24,9
...,...,...,...
109224741,353404,243,7
109224743,353404,392,9
109224744,353404,882,6
109224745,353404,883,8


In [34]:
user_df = user_df[user_df['rating'] != 0]
user_counts = user_df['user_id'].value_counts()

In [35]:
filtered_users_400 = user_counts[(user_counts > 400) & (user_counts <= 600)].index
sampled_users_400 = filtered_users_400.to_series().sample(n=150, random_state=42)
result_df_400 = user_df[user_df['user_id'].isin(sampled_users_400)]
result_df_400

,user_id,anime_id,rating
153808,516,40392,8
153809,516,39575,7
153810,516,41380,8
153811,516,36793,5
153812,516,38101,6
...,...,...,...
108160449,349955,15119,6
108160451,349955,40902,5
108160452,349955,3588,8
108160453,349955,849,8


In [36]:
filtered_users_100_200 = user_counts[user_counts.between(100, 200)].index
sampled_users_100_200 = filtered_users_100_200.to_series().sample(n=400, random_state=42)
result_df_100_200 = user_df[user_df['user_id'].isin(sampled_users_100_200)]
result_df_100_200

,user_id,anime_id,rating
226051,757,12291,10
226052,757,15085,8
226053,757,1689,8
226054,757,23333,7
226056,757,12189,9
...,...,...,...
108544892,351152,1726,7
108544893,351152,249,7
108544894,351152,33489,5
108544895,351152,1254,8


In [37]:
filtered_users_50_100 = user_counts[user_counts.between(50, 100)].index
sampled_users_50_100 = filtered_users_50_100.to_series().sample(n=500, random_state=42)
result_df_50_100 = user_df[user_df['user_id'].isin(sampled_users_50_100)]
result_df_50_100

,user_id,anime_id,rating
221116,739,18671,10
221121,739,457,7
221122,739,16067,7
221124,739,205,8
221129,739,47,9
...,...,...,...
109178967,353272,37976,6
109178982,353272,6500,7
109178983,353272,4063,6
109178987,353272,36882,2


In [38]:
final_result_df = pd.concat([result_df_400, result_df_100_200, result_df_50_100]).reset_index(drop=True)
final_result_df

,user_id,anime_id,rating
0,516,40392,8
1,516,39575,7
2,516,41380,8
3,516,36793,5
4,516,38101,6
...,...,...,...
166755,353272,37976,6
166756,353272,6500,7
166757,353272,4063,6
166758,353272,36882,2


In [39]:
final_result_df['user_id'].nunique()

1050

# Data evaluation and recommendation

In [40]:
"""def split_data(data, split_percentage):
    def split_group(group):
        split_index = int(len(group) * (1 - split_percentage))
        return group.iloc[split_index:], group.iloc[:split_index]
    
    grouped = data.groupby('user_id')
    
    train_list, test_list = zip(*grouped.apply(lambda x: split_group(x)).values)
    
    train_set = pd.concat(train_list).reset_index(drop=True)
    test_set = pd.concat(test_list).reset_index(drop=True)
    
    rating = list(train_set.rating)

    users = data['user_id'].unique()
    items = data['anime_id'].unique()

    rows = train_set.user_id.astype(pd.api.types.CategoricalDtype(categories = users)).cat.codes
    cols = train_set.anime_id.astype(pd.api.types.CategoricalDtype(categories = items)).cat.codes

    rating_matrix = sparse.csr_matrix((rating, (rows, cols)), shape = (len(users), len(items)))

    return rating_matrix, train_set, test_set"""

def split_data(data, split_percentage):
    def split_group(group):
        group = group.sample(frac=1, random_state=1).reset_index(drop=True)
        split_index = int(len(group) * split_percentage)
        return group.iloc[:split_index], group.iloc[split_index:]
    
    grouped = data.groupby('user_id')
    
    train_list, test_list = zip(*grouped.apply(lambda x: split_group(x)).values)
    
    train_set = pd.concat(train_list).reset_index(drop=True)
    test_set = pd.concat(test_list).reset_index(drop=True)
    
    train_user_ids = set(train_set['user_id'])
    test_user_ids = set(test_set['user_id'])
    
    users_in_both = train_user_ids.intersection(test_user_ids)
    
    if users_in_both != train_user_ids or users_in_both != test_user_ids:
        print("Warning: Some user_ids are not present in both datasets.")
    
    return train_set, test_set
    
def plot_similarity_matrix(similarity_matrix, similarity_metric, base):
    plt.figure()
    plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
    plt.title(f"{base}-{base} {similarity_metric} Similarity Matrix")
    plt.colorbar()

In [41]:
final_result_df['rating'].value_counts()

rating
8     44616
7     36644
9     31926
10    20893
6     16629
5      8393
4      3990
3      1885
2       976
1       808
Name: count, dtype: int64

In [42]:
user_id = 10
k_values = [5, 10, 15, 30, 50, 100]
threshold = 3.5
top_n = 10

split_percentage = 0.9              # 90% of the data will be used for training and 10% for testing
core = 1000                         # keep only items that where rated by at least $core$ users

In [43]:
#rating_matrix, train, test = split_data(final_result_df, split_percentage)
train, test = split_data(final_result_df, split_percentage)

/tmp/ipykernel_87574/3529723414.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_list, test_list = zip(*grouped.apply(lambda x: split_group(x)).values)


In [53]:
train['anime_id'].nunique()

3087

In [45]:
test['predicted_rating'] = None
test

,user_id,anime_id,rating,predicted_rating
0,516,39463,7,None
1,516,918,8,None
2,516,32899,5,None
3,516,38080,6,None
4,516,30016,6,None
...,...,...,...,...
17132,353272,13663,8,None
17133,353272,15583,9,None
17134,353272,31163,8,None
17135,353272,33010,6,None


## 3.2 Item-based Collaborative Filtering

### • Item Similarity Computation

In [46]:
interactions = final_result_df[['user_id', 'anime_id', 'rating']].values.tolist()

dataset = Dataset.from_uir(interactions, seed=42)


### • Recommendation Generation

In [47]:
dataset.matrix[516]

<1x3119 sparse matrix of type '<class 'numpy.float64'>'
	with 167 stored elements in Compressed Sparse Row format>

In [48]:
itemcf=ItemKNN(k=10)
itemcf.fit(dataset) # FEURRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR

100%|██████████| 3119/3119 [00:00<00:00, 21421.02it/s]


In [52]:
for index, row in test.iterrows():
    try:
        test.at[index, 'predicted_rating'] = itemcf.score(row['user_id'], row['anime_id'])
    except Exception as e:
        print(e)
        continue

Can't make score prediction for item 39463
Can't make score prediction for item 32899
Can't make score prediction for item 38080
Can't make score prediction for item 30016
Can't make score prediction for item 33206
Can't make score prediction for item 6675
Can't make score prediction for item 39575
Can't make score prediction for item 38414
Can't make score prediction for item 31821
Can't make score prediction for item 30015
Can't make score prediction for item 28497
Can't make score prediction for item 22199
Can't make score prediction for item 36094
Can't make score prediction for item 37347
Can't make score prediction for item 35847
Can't make score prediction for item 35062
Can't make score prediction for item 31952
Can't make score prediction for item 11843
Can't make score prediction for item 10719
Can't make score prediction for item 36884
Can't make score prediction for item 28817
Can't make score prediction for item 14813
Can't make score prediction for item 11757
Can't make s

In [ ]:
test = test.dropna(subset=['predicted_rating'])
test

,user_id,anime_id,rating,predicted_rating
1,516,918,8,9.7
43,739,889,8,7.501525
46,739,269,7,7.802829
49,757,687,7,8.803334
51,757,996,5,9.10065
52,757,2476,8,8.8
58,757,530,8,9.300559
59,757,763,1,7.996048
60,757,3092,9,9.5
61,767,97,10,8.500573


## 3.3 User-based Collaborative Filtering

### • Item Similarity Computation

### • Neighborhood Formation

In [ ]:
plot_similarity_matrix(usercf.sim_mat, 'cosine', 'User')

NameError: name 'usercf' is not defined

### • Recommendation Generation

In [ ]:
usercf.predict(user_id, top_n)